
# Modelos Avançados com Python

As **redes neurais e modelos supervisionados avançados** são amplamente utilizados em problemas de **classificação, regressão e reconhecimento de padrões complexos**.

Cada tipo de arquitetura possui **características específicas**, adequadas a determinados tipos de dados e objetivos.

| Modelo | Ideal para | Tipo de Dados | Pontos Fortes |
|:--|:--|:--|:--|
| **DNN (Deep Neural Network)** | Dados tabulares e complexos | Numéricos e categóricos | Captura relações não lineares e interações entre variáveis |
| **CNN (Convolutional Neural Network)** | Imagens, vídeos, sinais espaciais | Matriciais (pixels) | Capta padrões espaciais e visuais |
| **RNN (Recurrent Neural Network)** | Séries temporais e textos | Sequenciais | Capta dependências temporais e contextuais |
| **SVM (Support Vector Machine)** | Dados bem definidos e separáveis | Tabulares, numéricos | Eficiente em conjuntos menores e alta dimensionalidade |

## DNN — Redes Neurais Profundas

As **Deep Neural Networks (DNNs)** são compostas por **múltiplas camadas ocultas** de neurônios artificiais, permitindo a modelagem de **relações altamente não lineares**.

Cada camada aprende **representações progressivamente mais abstratas** dos dados de entrada.

**Vantagens:**
- Excelente desempenho em dados tabulares e numéricos.  
- Aprende relações complexas entre variáveis.  
- Flexível e de fácil implementação com frameworks como TensorFlow e Keras.

**Limitações:**
- Pode superajustar (overfitting) se os dados forem poucos.  
- Requer normalização e ajuste de hiperparâmetros.  

In [ ]:
# Exemplo Prático — Classificação de Câncer (Dataset: Breast Cancer)

import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Carrega dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Padronização
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criação da DNN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilação e treinamento
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=16, verbose=1)

# Avaliação
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Acurácia:", accuracy_score(y_test, y_pred))

## CNN — Redes Neurais Convolucionais


As **Convolutional Neural Networks (CNNs)** são projetadas para trabalhar com dados que possuem **estrutura espacial**, como imagens ou vídeos.  
Utilizam **camadas convolucionais** que aplicam **filtros (kernels)** para detectar padrões locais (bordas, texturas, formas).

**Componentes principais:**
- **Camada Convolucional:** extrai padrões locais.  
- **Camada de Pooling:** reduz a dimensionalidade e mantém características relevantes.  
- **Camada densa final:** realiza a classificação.

**Vantagens:**
- Reconhece padrões invariantes de posição.  
- Reduz a necessidade de engenharia manual de características.  
- Escalável e amplamente usada em visão computacional.

**Limitações:**
- Requer mais recursos computacionais.  
- Necessita grande volume de dados.  


In [ ]:
# Classificação de Dígitos (MNIST)

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normaliza e ajusta formato
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Criação da CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compilação e treinamento
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

# Avaliação
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Acurácia no teste:", test_acc)

## RNN — Redes Neurais Recorrentes

As **RNNs (Recurrent Neural Networks)** foram projetadas para processar **dados sequenciais**, onde a ordem das entradas é importante (ex.: séries temporais, textos).  
Elas possuem **conexões recorrentes** que permitem reter **informações anteriores** (memória).

**Variações comuns:**
- **LSTM (Long Short-Term Memory):** evita o problema de gradiente desaparecendo.  
- **GRU (Gated Recurrent Unit):** mais leve e eficiente que LSTM.

**Vantagens:**
- Captura dependências temporais.  
- Ideal para previsões em séries temporais e textos.

**Limitações:**
- Treinamento mais lento.  
- Dificuldade com dependências de longo prazo (parcialmente resolvido com LSTM/GRU).

In [ ]:
# Previsão de Série Temporal Sintética

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Geração de série temporal
time = np.arange(0, 200, 0.1)
data = np.sin(time) + 0.1 * np.random.randn(len(time))

# Normaliza
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.reshape(-1,1))

# Cria janelas (sequências)
def create_dataset(data, step=10):
    X, y = [], []
    for i in range(len(data)-step):
        X.append(data[i:i+step])
        y.append(data[i+step])
    return np.array(X), np.array(y)

X, y = create_dataset(data_scaled, step=20)
X_train, y_train = X[:1500], y[:1500]
X_test, y_test = X[1500:], y[1500:]

# Criação da RNN LSTM
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='tanh', input_shape=(20,1)),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=1)

# Avaliação
pred = model.predict(X_test)
plt.plot(y_test, label='Real')
plt.plot(pred, label='Previsto')
plt.legend()
plt.show()

## SVM — Máquinas de Vetores de Suporte

O **SVM (Support Vector Machine)** busca um **hiperplano ótimo** que separa as classes com **máxima margem**.  
Trabalha bem mesmo em **alta dimensionalidade**, utilizando **funções kernel** para transformar dados não lineares em um espaço separável.

**Principais kernels:**
- Linear  
- Polinomial (`poly`)  
- Radial Basis Function (`rbf`)  

**Vantagens:**
- Eficiente em conjuntos pequenos e bem estruturados.  
- Alta precisão com ajustes adequados.

**Limitações:**
- Custo computacional alto para datasets grandes.  
- Difícil ajuste de hiperparâmetros `C` e `gamma`.

In [ ]:
# Classificação no Iris Dataset

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

iris = datasets.load_iris()
X, y = iris.data, iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = SVC(kernel='rbf', C=1.0, gamma='scale')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Acurácia:", accuracy_score(y_test, y_pred))
ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=iris.target_names).plot()
plt.show()

## Exercícios Propostos

1. **DNN:**  
   - Aplique a rede ao dataset *Wine* (sklearn).  
   - Teste 2, 3 e 4 camadas ocultas.  
   - Compare tempo e acurácia.

2. **CNN:**  
   - Use o dataset *CIFAR-10*.  
   - Compare o desempenho alterando filtros e camadas.

3. **RNN:**  
   - Gere uma série temporal com tendência crescente.  
   - Compare desempenho de LSTM vs GRU.

4. **SVM:**  
   - Compare os kernels `'linear'`, `'poly'` e `'rbf'`.  
   - Explique visualmente o impacto do parâmetro `C`.

## Conclusão

| Modelo | Melhor uso | Complexidade | Vantagens |
|:--|:--|:--|:--|
| **DNN** | Dados tabulares | Média | Flexível e adaptável |
| **CNN** | Imagens | Alta | Extração automática de padrões visuais |
| **RNN** | Séries temporais | Alta | Capta dependências temporais |
| **SVM** | Dados estruturados pequenos | Média | Alta precisão com poucos dados |